<a href="https://colab.research.google.com/github/HariPrabu741/RNN/blob/main/Positive_Negative_reviews.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget https://www.dropbox.com/s/pdhwlpi2yeie0ol/movie-reviews-dataset.zip

--2021-09-02 02:11:45--  https://www.dropbox.com/s/pdhwlpi2yeie0ol/movie-reviews-dataset.zip
Resolving www.dropbox.com (www.dropbox.com)... 162.125.64.18, 2620:100:6020:18::a27d:4012
Connecting to www.dropbox.com (www.dropbox.com)|162.125.64.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/pdhwlpi2yeie0ol/movie-reviews-dataset.zip [following]
--2021-09-02 02:11:45--  https://www.dropbox.com/s/raw/pdhwlpi2yeie0ol/movie-reviews-dataset.zip
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc434e5bb18c784494a72418ff4e.dl.dropboxusercontent.com/cd/0/inline/BVWq3dykkbEbbiW7u7gutTiscHJ_kYs3F2kPZY575Rn_E4CcJp-chNsaa7d0iKLO--kwcSMVj8t4ELdolK91W8nSyboWaitY6o3f1iij-3Mmv3VcwaweGnk9XFTLtbf3kMT0d7qx75PVKaLRxdi5u4Mq/file# [following]
--2021-09-02 02:11:46--  https://uc434e5bb18c784494a72418ff4e.dl.dropboxusercontent.com/cd/0/inline/BVWq3dykkbEbbiW7u7gutTiscHJ_kYs3F2kPZY575Rn_E4Cc

In [3]:
!unzip -q '/content/movie-reviews-dataset.zip'

In [4]:
from tensorflow.keras.preprocessing import text_dataset_from_directory
from tensorflow.strings import regex_replace
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
from tensorflow.keras.models import Sequential
from tensorflow.keras import Input
from tensorflow.keras.layers import Dense,RNN,Dropout,Embedding,SimpleRNNCell

In [5]:
def prepare_data(dir):
  data=text_dataset_from_directory(dir)
  return data.map(lambda text,label:(regex_replace(text,'<br />',''),label),)

In [6]:
train_data=prepare_data("/content/movie-reviews-dataset/train")
test_data=prepare_data('/content/movie-reviews-dataset/test')

Found 25000 files belonging to 2 classes.
Found 25000 files belonging to 2 classes.


In [10]:
for text_batch,label_batch in train_data.take(1):
  print(text_batch.numpy()[0])
  print(label_batch.numpy()[0])

b"A great concept, a great cast, and what a pity there wasn't more time to flesh out the story. I loved it and wanted more. Dench, Dukakis, and Laine, now there are some REAL women! Still, Dench and her character alone had enough substance to carry the script over some of its lesser moments. I have it on tape and will continue to watch it, hoping that there is a clue at the end that suggests there will be a sequel.Top drawer! - No Question! - No Argument!"
1


In [11]:
model=Sequential()

In [12]:
model.add(Input(shape=(1,),dtype='string'))

In [13]:
max_tokens=1000
max_len=100
vectorize_layer=TextVectorization(max_tokens=max_tokens,output_mode='int',output_sequence_length=max_len)

In [14]:
train_texts=train_data.map(lambda text,label:text)
vectorize_layer.adapt(train_texts)
model.add(vectorize_layer)

In [15]:
model.add(Embedding(max_tokens+1,128))
rnn=RNN(SimpleRNNCell(64),return_sequences=False,return_state=False)
model.add(rnn)
model.add(Dense(64,activation='relu'))
model.add(Dense(1,activation='sigmoid'))

In [16]:
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [17]:
model.fit(train_data,epochs=10)

Epoch 1/10
782/782 [==============================] - 107s 132ms/step - loss: 0.6951 - accuracy: 0.5103
Epoch 2/10
782/782 [==============================] - 103s 131ms/step - loss: 0.6716 - accuracy: 0.5774
Epoch 3/10
782/782 [==============================] - 103s 132ms/step - loss: 0.6257 - accuracy: 0.6454
Epoch 4/10
782/782 [==============================] - 104s 132ms/step - loss: 0.6683 - accuracy: 0.5848
Epoch 5/10
782/782 [==============================] - 104s 132ms/step - loss: 0.6505 - accuracy: 0.6089
Epoch 6/10
782/782 [==============================] - 104s 133ms/step - loss: 0.6172 - accuracy: 0.6502
Epoch 7/10
782/782 [==============================] - 103s 132ms/step - loss: 0.6451 - accuracy: 0.6208
Epoch 8/10
782/782 [==============================] - 104s 133ms/step - loss: 0.6407 - accuracy: 0.6142
Epoch 9/10
782/782 [==============================] - 103s 132ms/step - loss: 0.6184 - accuracy: 0.6459
Epoch 10/10
782/782 [==============================] - 101s 129m

In [18]:
model.evaluate(test_data)

782/782 [==============================] - 17s 22ms/step - loss: 0.7167 - accuracy: 0.5462


[0.7167445421218872, 0.546239972114563]

In [19]:
text='I love the movie'

In [20]:
model.predict([text])  # If value is more than 0.50 it is positive review 

array([[0.5952416]], dtype=float32)

In [21]:
texts='Not nice'

In [22]:
model.predict([texts])  # If value is less than 0.50 it is negative review

array([[0.461339]], dtype=float32)